**SET UP VÀ CÀI ĐẶT THƯ VIỆN**

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as F # Vẫn giữ F vì có thể dùng F.to_tensor
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
import os
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
from torch.amp import autocast, GradScaler
import numpy as np
import matplotlib.pyplot as plt

**ĐƯỜNG DẪN**

In [ ]:
COCO_DATASET_BASE_PATH = '/kaggle/input/vietnam-traffic-sign-coco/faster_rcnn_coco_dataset' 
LOAD_EPOCH = 5 # 
model_path_to_load = f'/kaggle/input/mycheckpoint/faster_rcnn_model_epoch_{LOAD_EPOCH}.pth'

**HYPERPARAMETER VÀ TRAIN MODEL**

In [ ]:
# --- Cấu hình chung ---
NUM_CLASSES = 29 + 1 
EPOCHS = 15 # 
BATCH_SIZE = 4 
LEARNING_RATE = 0.001
MOMENTUM = 0.97
WEIGHT_DECAY = 0.0005
WARMUP_EPOCHS = 3 # Số epoch dùng cho warm-up
WARMUP_FACTOR = 1/3 # Tốc độ học khởi đầu = LEARNING_RATE * WARMUP_FACTOR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

TRAIN_ANNOTATIONS_FILE = os.path.join(COCO_DATASET_BASE_PATH, 'annotations', 'instances_train.json')
VAL_ANNOTATIONS_FILE = os.path.join(COCO_DATASET_BASE_PATH, 'annotations', 'instances_val.json')
IMAGES_DIR = os.path.join(COCO_DATASET_BASE_PATH, 'images')

# --- Lớp Dataset Tùy chỉnh cho COCO ---
class CustomCOCODataset(torchvision.datasets.CocoDetection):
    def __init__(self, root, annFile, is_train=True):
        super().__init__(root, annFile)
        self.is_train = is_train

        # Định nghĩa các phép biến đổi torchvision.transforms
        # Gợi ý tối ưu: Thêm các phép tăng cường dữ liệu ở đây cho chế độ huấn luyện
        if self.is_train:
            self.transform = T.Compose([
                T.ToTensor(),
            ])
        else:
            self.transform = T.Compose([
                T.ToTensor(),
            ])

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_pil, target_list_of_dicts = super().__getitem__(idx)

        # Áp dụng torchvision.transforms cho ảnh
        img_tensor = self.transform(img_pil)

        valid_boxes = []
        valid_labels = []
        for ann in target_list_of_dicts:
            bbox = ann['bbox']
            x_min, y_min, w, h = bbox
            x_max = x_min + w
            y_max = y_min + h

            if w > 0 and h > 0 and ann['area'] > 0:
                valid_boxes.append([x_min, y_min, x_max, y_max]) # Pascal VOC format
                valid_labels.append(ann['category_id'])

        boxes_transformed = torch.tensor(valid_boxes, dtype=torch.float32)
        labels_transformed = torch.tensor(valid_labels, dtype=torch.int64)

        if boxes_transformed.numel() == 0:
            boxes_transformed = torch.zeros((0, 4), dtype=torch.float32)
            labels_transformed = torch.zeros((0,), dtype=torch.int64)

        image_id = torch.tensor([img_id])
        area = (boxes_transformed[:, 3] - boxes_transformed[:, 1]) * \
               (boxes_transformed[:, 2] - boxes_transformed[:, 0])
        iscrowd = torch.zeros((len(boxes_transformed),), dtype=torch.int64)

        target_frcnn = {}
        target_frcnn["boxes"] = boxes_transformed
        target_frcnn["labels"] = labels_transformed
        target_frcnn["image_id"] = image_id
        target_frcnn["area"] = area
        target_frcnn["iscrowd"] = iscrowd

        return img_tensor, target_frcnn

# Hàm collate_fn vẫn giữ nguyên
def collate_fn(batch):
    return tuple(zip(*batch))

# Khởi tạo Dataset và DataLoader
train_dataset = CustomCOCODataset(root=IMAGES_DIR,
                                  annFile=TRAIN_ANNOTATIONS_FILE,
                                  is_train=True)

val_dataset = CustomCOCODataset(root=IMAGES_DIR,
                                annFile=VAL_ANNOTATIONS_FILE,
                                is_train=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

# --- Hàm hỗ trợ tính toán mAP bằng pycocotools ---
def evaluate_coco(model, data_loader, coco_gt, device):
    model.eval()
    results = []

    for images, targets in tqdm(data_loader, desc="Collecting Predictions"):
        images = list(img.to(device) for img in images)

        batch_image_ids = [t["image_id"].item() for t in targets]

        with autocast(device_type='cuda'):
            outputs = model(images)

        for i, output in enumerate(outputs):
            image_id = batch_image_ids[i]

            boxes = output["boxes"].detach().cpu().numpy()
            labels = output["labels"].detach().cpu().numpy()
            scores = output["scores"].detach().cpu().numpy()

            for box, label, score in zip(boxes, labels, scores):
                x_min, y_min, x_max, y_max = box
                width = x_max - x_min
                height = y_max - y_min

                results.append({
                    "image_id": int(image_id),
                    "category_id": int(label),
                    "bbox": [float(x_min), float(y_min), float(width), float(height)],
                    "score": float(score)
                })

    temp_results_file = "temp_results.json"
    with open(temp_results_file, "w") as f:
        json.dump(results, f)

    if not results:
        print("Không có dự đoán nào được tạo ra. Không thể tính toán mAP.")
        return 0.0, 0.0

    coco_dt = coco_gt.loadRes(temp_results_file)

    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")

    coco_eval.params.imgIds = coco_gt.getImgIds()

    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    mAP = coco_eval.stats[0]
    mAP_50 = coco_eval.stats[1]

    os.remove(temp_results_file)

    return mAP, mAP_50

# --- Tải Mô hình Faster R-CNN ---
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

# --- Định nghĩa Optimizer và Learning Rate Scheduler ---
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

warmup_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=WARMUP_FACTOR, total_iters=WARMUP_EPOCHS)
main_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

lr_scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[warmup_scheduler, main_scheduler], milestones=[WARMUP_EPOCHS])

scaler = GradScaler()

# --- Tải mô hình đã lưu để tiếp tục huấn luyện ---

if os.path.exists(model_path_to_load) and LOAD_EPOCH > 0:
    model.load_state_dict(torch.load(model_path_to_load, map_location=device))
    print(f"Đã tải mô hình từ: {model_path_to_load} để tiếp tục huấn luyện.")
    for _ in range(LOAD_EPOCH):
        lr_scheduler.step()
else:
    print(f"Không tìm thấy file mô hình tại {model_path_to_load} hoặc LOAD_EPOCH là 0. Bắt đầu huấn luyện từ đầu.")

model.to(device)
model.train()

# --- Danh sách để lưu trữ các chỉ số qua các epoch ---
train_losses = []
val_mAP_50_95 = []
val_mAP_50 = []
epochs_ran = []
print("\n--- Bắt đầu huấn luyện Faster R-CNN ---")
coco_val_gt = COCO(VAL_ANNOTATIONS_FILE)

for epoch in range(LOAD_EPOCH, EPOCHS):
    current_epoch_display = epoch + 1
    epochs_ran.append(current_epoch_display)

    model.train()
    total_loss = 0

    print(f"\nEpoch {current_epoch_display}/{EPOCHS}")
    for i, (images, targets) in enumerate(tqdm(train_loader, desc="Training")):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        with autocast(device_type='cuda'):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += losses.item()

    lr_scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    print(f"Loss Epoch {current_epoch_display}: {avg_train_loss:.4f}")

    # --- Đánh giá trên tập Validation (có tính mAP) ---
    print("\n--- Bắt đầu đánh giá trên tập Validation ---")
    model.eval()
    with torch.no_grad():
        # Không cần vòng lặp ở đây, evaluate_coco sẽ tự gọi model(images)
        pass 

    print(f"\n--- Tính toán mAP cho Epoch {current_epoch_display} ---")
    mAP_val, mAP_50_val = evaluate_coco(model, val_loader, coco_val_gt, device)
    val_mAP_50_95.append(mAP_val)
    val_mAP_50.append(mAP_50_val)
    print(f"mAP (IoU=0.50:0.95): {mAP_val:.4f}")
    print(f"mAP@0.50: {mAP_50_val:.4f}")

    # --- Lưu mô hình sau mỗi epoch ---
    model_save_path = f"faster_rcnn_model_epoch_{current_epoch_display}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Đã lưu mô hình tại: {model_save_path}")

print("\n--- Huấn luyện Faster R-CNN hoàn tất ---")

**VISUALIZATION**

In [ ]:
plt.figure(figsize=(12, 6))

# Biểu đồ Loss
plt.subplot(1, 2, 1)
plt.plot(epochs_ran, train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss per Epoch')
plt.legend()
plt.grid(True)

# Biểu đồ mAP
plt.subplot(1, 2, 2)
plt.plot(epochs_ran, val_mAP_50_95, label='mAP (IoU=0.50:0.95)')
plt.plot(epochs_ran, val_mAP_50, label='mAP@0.50')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('Validation mAP per Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()